In [32]:
from typing import List
import itertools
import json
from copy import deepcopy

In [4]:
dist = 'decision_tree/households_cars_dist'
change_value = 0.2

In [33]:
dd = {
    'a': {
        '1': 0.2,
        '2': 0.3,
        '3': 0.5
    },
    'b': {
        '1': 0.2,
        '2': 0.3,
        '3': 0.5
    }
}

In [42]:
def change_dist(dist, value):

    if value != 0:

        if value < 0:
            value = abs(value)
            reverse = False
        else:
            reverse = True

        for dist_key in dist.keys():
            d = dist[dist_key]
            prev_key = None
            for k in sorted(d.keys(), reverse=reverse):
                if prev_key:
                    change_value = 0.1 * d[k]
                    d[prev_key] = d[prev_key] + change_value
                    d[k] = d[k] - change_value
                prev_key = k
            dist[dist_key] = d

    return dist

In [43]:
change_dist(deepcopy(dd), 0.2)

{'a': {'1': 0.18, '2': 0.29000000000000004, '3': 0.53},
 'b': {'1': 0.18, '2': 0.29000000000000004, '3': 0.53}}

In [44]:
change_dist(deepcopy(dd), -0.2)

{'a': {'1': 0.23, '2': 0.32, '3': 0.45},
 'b': {'1': 0.23, '2': 0.32, '3': 0.45}}

In [45]:
dd

{'a': {'1': 0.2, '2': 0.3, '3': 0.5}, 'b': {'1': 0.2, '2': 0.3, '3': 0.5}}

In [38]:
sorted(dd['a'].keys(), reverse=False)

['1', '2', '3']

In [48]:
sc = [["tr/f1", 0.1], ["tr/f2", 0.1], ["tr/f3", 0.1]]

In [54]:
def get_output_folder_postfix(scenario):
    postfix = ''

    for [dist, value] in scenario:
        postfix += '_'
        postfix += dist.split('/')[-1]
        postfix += '_'
        postfix += str(value).replace('.', '_')

    return postfix

In [55]:
get_output_folder_postfix(sc)

'_f1_0_1_f2_0_1_f3_0_1'

In [2]:
raise Exception('aa')

Exception: aa